In [1]:
from PIL import Image
import os
import PIL
import pandas as pd

In [2]:
INPUT_DIR = '../input/'


In [3]:
TRAIN_IMG_PATH = os.path.join(INPUT_DIR, 'train')
TEST_IMG_PATH = os.path.join(INPUT_DIR, 'test')

TRAIN_CROP = os.path.join(INPUT_DIR, 'train_crop_299')
TEST_CROP = os.path.join(INPUT_DIR, 'test_crop_299')

TRAIN_CROP_RATIO = os.path.join(INPUT_DIR, 'train_crop_299_ratio')
TEST_CROP_RATIO = os.path.join(INPUT_DIR, 'test_crop_299_ratio')


df_train = pd.read_csv(os.path.join(INPUT_DIR, 'train.csv'))
df_test = pd.read_csv(os.path.join(INPUT_DIR, 'test.csv'))

In [4]:
#fix size with ratio
def fix_size(im, desired_w, desired_h, fill_color=(0, 0, 0, 255)):
    x, y = im.size

    ratio = x / y
    desired_ratio = desired_w / desired_h

    w = max(desired_w, x)
    h = int(w / desired_ratio)
    if h < y:
        h = y
        w = int(h * desired_ratio)

    new_im = Image.new('RGB', (w, h), fill_color)
    new_im.paste(im, ((w - x) // 2, (h - y) // 2))
    return new_im.resize((desired_w, desired_h))

In [5]:
#crop image no ratio 
def crop_boxing_img(row, path, size,  margin=0, ratio=False):

    img = PIL.Image.open(os.path.join(path,row['img_file']))
    pos =row[['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2']].values.reshape(-1)
    width, height = img.size
    
    x1 = max(0, pos[0] - margin)
    y1 = max(0, pos[1] - margin)
    x2 = min(pos[2] + margin, width)
    y2 = min(pos[3] + margin, height)
    
    img_no_ratio = img.crop((x1, y1, x2, y2)).resize((size,size))
    img_ratio = None
    if ratio==True:
        img_ratio = fix_size(img.crop((x1,y1,x2,y2)),
                             desired_w=size,
                             desired_h=size)
    return img_no_ratio, img_ratio
 

In [6]:
'''
import shutil
shutil.rmtree(TRAIN_CROP, ignore_errors=True)
shutil.rmtree(TEST_CROP, ignore_errors=True)
shutil.rmtree(TRAIN_CROP_RATIO, ignore_errors=True)
shutil.rmtree(TEST_CROP_RATIO, ignore_errors=True)
'''

'\nimport shutil\nshutil.rmtree(TRAIN_CROP, ignore_errors=True)\nshutil.rmtree(TEST_CROP, ignore_errors=True)\nshutil.rmtree(TRAIN_CROP_RATIO, ignore_errors=True)\nshutil.rmtree(TEST_CROP_RATIO, ignore_errors=True)\n'

In [7]:
def crop_img(df, path_image, path_crop, path_crop_ratio, image_size, ratio=False):
    if not os.path.exists(path_crop):
        os.mkdir(path_crop)
        if ratio:
            os.mkdir(path_crop_ratio)
        print('>>mkdir {}'.format(path_crop))
        print('>>mkdir {}'.format(path_crop_ratio))
        for i, row in df.iterrows():
            cropped, cropped_ratio = crop_boxing_img(row,
                                             path_image,
                                             size=image_size,
                                             ratio=ratio)
            cropped.save(path_crop+"/"+row['img_file'])
            if ratio:
                cropped_ratio.save(path_crop_ratio+"/"+row['img_file'])
        print('>>crop completed')
    else:
        print('>>{} exist'.format(path_crop))

In [8]:
image_size=299

crop_img(df=df_train,
         path_image=TRAIN_IMG_PATH,
         path_crop=TRAIN_CROP,
         path_crop_ratio=TRAIN_CROP_RATIO,
         image_size=image_size,
         ratio=False)

crop_img(df=df_test,
         path_image=TEST_IMG_PATH,
         path_crop=TEST_CROP,
         path_crop_ratio=TEST_CROP_RATIO,
         image_size=image_size,
         ratio=False)

>>mkdir ../input/train_crop_299
>>mkdir ../input/train_crop_299_ratio
>>crop completed
>>mkdir ../input/test_crop_299
>>mkdir ../input/test_crop_299_ratio
>>crop completed


In [9]:
print(len(os.listdir(TRAIN_CROP)))
print(len(os.listdir(TEST_CROP)))


9990


FileNotFoundError: [Errno 2] No such file or directory: '../input/train_crop_299_ratio'

In [ ]:
import matplotlib
%matplotlib inline
PIL.Image.open(os.path.join(TRAIN_CROP, os.listdir(TRAIN_CROP)[0]))

In [ ]:
PIL.Image.open(os.path.join(TRAIN_CROP_RATIO, os.listdir(TRAIN_CROP_RATIO)[0]))

In [ ]:

TRAIN_CROP_224 = os.path.join(INPUT_DIR, 'train_crop_224')
TEST_CROP_224 = os.path.join(INPUT_DIR, 'test_crop_224')
TRAIN_CROP_224_RATIO = os.path.join(INPUT_DIR, 'train_crop_224_ratio')
TEST_CROP_224_RATIO = os.path.join(INPUT_DIR, 'test_crop_224_ratio')

image_size=224

crop_img(df=df_train,
         path_image=TRAIN_IMG_PATH,
         path_crop=TRAIN_CROP_224,
         path_crop_ratio=TRAIN_CROP_224_RATIO,
         image_size=image_size)

crop_img(df=df_test,
         path_image=TEST_IMG_PATH,
         path_crop=TEST_CROP_224,
         path_crop_ratio=TEST_CROP_224_RATIO,
         image_size=image_size)